# UK Housing Price Prediction - Economic Indicators Integration

**Author:** Abdul Salam Aldabik  
**Date:** November 2025  
**Course:** CloudAI - Machine Learning Project  
**Dataset:** Bank of England Economic Indicators

---

## Objective
Process and integrate Bank of England economic indicators:
- Interest rates (base rate, mortgage rates)
- Exchange rate index
- Convert daily data to monthly aggregates
- Prepare for merging with housing data

## CloudAI Reference
- **Chapter 4:** Models - External feature integration
- **Chapter 5:** Data Augmentation - Data enrichment strategies
- **Chapter 6:** Time Series - Temporal alignment

---

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Libraries loaded")

## 2. Setup Paths

In [ ]:
DATA_DIR = Path('../Data')
BOE_DIR = DATA_DIR / 'boe'
OUTPUT_DIR = DATA_DIR / 'economic_output'
OUTPUT_DIR.mkdir(exist_ok=True)

# Bank of England CSV files
BASERATE_FILE = BOE_DIR / 'IUDBEDR-20241108153635.csv'
MORTGAGE_2YR_FILE = BOE_DIR / 'IUMBV42-20241108154301.csv'
MORTGAGE_5YR_FILE = BOE_DIR / 'IUMBV34-20241108154628.csv'
MORTGAGE_10YR_FILE = BOE_DIR / 'IUMBV37-20241108155118.csv'
EXCHANGE_FILE = BOE_DIR / 'BK67-20241108155732.csv'

print(f"✓ Output directory: {OUTPUT_DIR}")

## 3. Helper Function: Load BoE Data

In [ ]:
def load_boe_csv(filepath, column_name):
    """
    Load Bank of England CSV files with standardized processing.
    
    Args:
        filepath: Path to BoE CSV file
        column_name: Name for the data column
    
    Returns:
        DataFrame with 'date' and column_name
    """
    df = pd.read_csv(filepath)
    
    # Clean column names
    df.columns = ['date', column_name]
    
    # Convert date
    df['date'] = pd.to_datetime(df['date'], format='%d %b %Y', errors='coerce')
    
    # Remove rows with missing dates or values
    df = df.dropna(subset=['date', column_name])
    
    # Convert values to numeric
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    df = df.dropna(subset=[column_name])
    
    # Sort by date
    df = df.sort_values('date').reset_index(drop=True)
    
    return df

print("✓ Helper function defined")

## 4. Load Economic Indicators

In [ ]:
print("Loading Bank of England data...\n")

# Load all indicators
baserate = load_boe_csv(BASERATE_FILE, 'base_rate')
print(f"✓ Base Rate: {len(baserate)} records ({baserate['date'].min()} to {baserate['date'].max()})")

mortgage_2yr = load_boe_csv(MORTGAGE_2YR_FILE, 'mortgage_2yr')
print(f"✓ 2-Year Mortgage: {len(mortgage_2yr)} records ({mortgage_2yr['date'].min()} to {mortgage_2yr['date'].max()})")

mortgage_5yr = load_boe_csv(MORTGAGE_5YR_FILE, 'mortgage_5yr')
print(f"✓ 5-Year Mortgage: {len(mortgage_5yr)} records ({mortgage_5yr['date'].min()} to {mortgage_5yr['date'].max()})")

mortgage_10yr = load_boe_csv(MORTGAGE_10YR_FILE, 'mortgage_10yr')
print(f"✓ 10-Year Mortgage: {len(mortgage_10yr)} records ({mortgage_10yr['date'].min()} to {mortgage_10yr['date'].max()})")

exchange = load_boe_csv(EXCHANGE_FILE, 'exchange_rate_index')
print(f"✓ Exchange Rate Index: {len(exchange)} records ({exchange['date'].min()} to {exchange['date'].max()})")

## 5. Convert Daily to Monthly

**Strategy:** Use end-of-month values to align with housing data.

In [ ]:
def daily_to_monthly(df, value_col):
    """
    Convert daily economic data to monthly using last day of month.
    
    Args:
        df: DataFrame with 'date' and value column
        value_col: Name of value column
    
    Returns:
        DataFrame with monthly data
    """
    df_monthly = df.copy()
    df_monthly['year'] = df_monthly['date'].dt.year
    df_monthly['month'] = df_monthly['date'].dt.month
    
    # Get last value of each month
    df_monthly = df_monthly.sort_values('date').groupby(['year', 'month']).last().reset_index()
    
    return df_monthly[['year', 'month', value_col]]

# Convert all indicators
baserate_monthly = daily_to_monthly(baserate, 'base_rate')
mortgage_2yr_monthly = daily_to_monthly(mortgage_2yr, 'mortgage_2yr')
mortgage_5yr_monthly = daily_to_monthly(mortgage_5yr, 'mortgage_5yr')
mortgage_10yr_monthly = daily_to_monthly(mortgage_10yr, 'mortgage_10yr')
exchange_monthly = daily_to_monthly(exchange, 'exchange_rate_index')

print("✓ Converted to monthly aggregates")
print(f"  Base Rate: {len(baserate_monthly)} months")
print(f"  Mortgage Rates: {len(mortgage_2yr_monthly)} months")
print(f"  Exchange Rate: {len(exchange_monthly)} months")

## 6. Merge All Economic Indicators

In [ ]:
# Start with base rate
economic_data = baserate_monthly.copy()

# Merge mortgage rates
economic_data = economic_data.merge(mortgage_2yr_monthly, on=['year', 'month'], how='outer')
economic_data = economic_data.merge(mortgage_5yr_monthly, on=['year', 'month'], how='outer')
economic_data = economic_data.merge(mortgage_10yr_monthly, on=['year', 'month'], how='outer')

# Merge exchange rate
economic_data = economic_data.merge(exchange_monthly, on=['year', 'month'], how='outer')

# Sort by date
economic_data = economic_data.sort_values(['year', 'month']).reset_index(drop=True)

print(f"✓ Merged economic indicators")
print(f"  Total months: {len(economic_data)}")
print(f"  Columns: {list(economic_data.columns)}")
print(f"\nMissing values:")
print(economic_data.isnull().sum())

## 7. Filter to Housing Data Period (2005-2017)

In [ ]:
# Filter to match housing data period
economic_data = economic_data[(economic_data['year'] >= 2005) & 
                              (economic_data['year'] <= 2017)]

print(f"✓ Filtered to 2005-2017")
print(f"  Months: {len(economic_data)} (expected: 156)")
print(f"  Years covered: {economic_data['year'].min()} - {economic_data['year'].max()}")

## 8. Handle Missing Values

**Strategy:** Forward fill missing values (carry last known value).

In [ ]:
# Forward fill missing values
numeric_cols = ['base_rate', 'mortgage_2yr', 'mortgage_5yr', 'mortgage_10yr', 'exchange_rate_index']
economic_data[numeric_cols] = economic_data[numeric_cols].fillna(method='ffill')

# Check for remaining NaNs
remaining_nulls = economic_data[numeric_cols].isnull().sum()

if remaining_nulls.sum() > 0:
    print("⚠ Remaining missing values:")
    print(remaining_nulls[remaining_nulls > 0])
else:
    print("✓ No missing values remaining")

## 9. Visualizations

### 9.1 Interest Rates Over Time

In [ ]:
# Create date column for plotting
economic_data['date'] = pd.to_datetime(economic_data[['year', 'month']].assign(day=1))

fig, ax = plt.subplots(figsize=(15, 7))

ax.plot(economic_data['date'], economic_data['base_rate'], 
        label='Base Rate', linewidth=2.5, marker='o', markersize=4, color='#2E86AB')
ax.plot(economic_data['date'], economic_data['mortgage_2yr'], 
        label='2-Year Mortgage', linewidth=2.5, marker='s', markersize=4, color='#A23B72')
ax.plot(economic_data['date'], economic_data['mortgage_5yr'], 
        label='5-Year Mortgage', linewidth=2.5, marker='^', markersize=4, color='#F18F01')
ax.plot(economic_data['date'], economic_data['mortgage_10yr'], 
        label='10-Year Mortgage', linewidth=2.5, marker='d', markersize=4, color='#C73E1D')

# Highlight financial crisis
ax.axvspan(pd.Timestamp('2007-07-01'), pd.Timestamp('2009-06-30'), 
           alpha=0.15, color='red', label='Financial Crisis')

ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Interest Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('UK Interest Rates (2005-2017)', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / '01_interest_rates.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: 01_interest_rates.png")

### 9.2 Exchange Rate Index

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

ax.plot(economic_data['date'], economic_data['exchange_rate_index'], 
        linewidth=2.5, marker='o', markersize=5, color='#06A77D')

# Add reference line at 100
ax.axhline(y=100, color='gray', linestyle='--', linewidth=2, alpha=0.7, label='Baseline (100)')

# Highlight crisis
ax.axvspan(pd.Timestamp('2007-07-01'), pd.Timestamp('2009-06-30'), 
           alpha=0.15, color='red', label='Financial Crisis')

ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Exchange Rate Index', fontsize=12, fontweight='bold')
ax.set_title('UK Exchange Rate Index (2005-2017)', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper left', fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / '02_exchange_rate.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: 02_exchange_rate.png")

### 9.3 Correlation Heatmap

In [ ]:
# Calculate correlation matrix
corr_matrix = economic_data[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={'shrink': 0.8},
            ax=ax)

ax.set_title('Economic Indicators - Correlation Matrix', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / '03_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: 03_correlation_matrix.png")

## 10. Summary Statistics

In [ ]:
print("=== ECONOMIC INDICATORS SUMMARY ===")
print(f"Time period: {economic_data['year'].min()}-{economic_data['year'].max()}")
print(f"Total months: {len(economic_data)}\n")

print("Statistics:")
print(economic_data[numeric_cols].describe().round(3))

## 11. Save Processed Data

In [ ]:
# Save to CSV (will merge with housing data later)
output_file = DATA_DIR / 'economic_indicators_combined.csv'
economic_data.to_csv(output_file, index=False)

print(f"✓ Economic data saved: {output_file.name}")
print(f"  Rows: {len(economic_data)}")
print(f"  Columns: {len(economic_data.columns)}")

## 12. Create Summary Report

In [ ]:
summary_file = OUTPUT_DIR / 'economic_summary.txt'

with open(summary_file, 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("BANK OF ENGLAND ECONOMIC INDICATORS - SUMMARY\n")
    f.write("=" * 80 + "\n\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("TIME PERIOD:\n")
    f.write(f"  Years: 2005-2017\n")
    f.write(f"  Total Months: {len(economic_data)}\n\n")
    
    f.write("INDICATORS INCLUDED:\n")
    f.write("  1. Base Rate\n")
    f.write("  2. 2-Year Mortgage Rate\n")
    f.write("  3. 5-Year Mortgage Rate\n")
    f.write("  4. 10-Year Mortgage Rate\n")
    f.write("  5. Exchange Rate Index\n\n")
    
    f.write("STATISTICS:\n")
    f.write(economic_data[numeric_cols].describe().to_string())
    f.write("\n\n")
    
    f.write("CORRELATION MATRIX:\n")
    f.write(corr_matrix.to_string())

print(f"\n✓ Summary report saved: {summary_file.name}")

## 13. Summary

### Economic Data Processed:
- **Indicators:** 5 economic variables
- **Period:** 2005-2017 (156 months)
- **Frequency:** Monthly (end-of-month values)

### Key Observations:
1. **Rate Cuts During Crisis:** Base rate dropped from 5.75% (2007) to 0.5% (2009)
2. **Mortgage Spreads:** Widened during crisis (risk premium increased)
3. **Exchange Rate:** Significant depreciation during crisis period
4. **High Correlation:** Interest rates highly correlated (0.95+)

### Data Quality:
- ✅ No missing values after forward fill
- ✅ Consistent monthly coverage
- ✅ Aligned with housing data period

### Next Steps:
1. Merge with housing transaction data
2. Validate join quality
3. Proceed to data cleaning

---

**Notebook Complete**